<a href="https://colab.research.google.com/github/Songhyunjeong/thingplug-lora-starter-kit/blob/master/notebooks/Learning_with_Gemini_and_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Use Gemini and ChatGPT to learn from two capable teachers</b>

Use Google's latest model release, Gemini, to teach you what you want to know and compare those with ChatGPT's responses.

The models are specifically prompted not to generate extra text to make it easier to compare any differences.

In [4]:
#@title Configure Gemini API key

# access your Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'G_Secret_key'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

model = genai.GenerativeModel('gemini-pro')

# 새 섹션

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [13]:
from openai import OpenAI
from google.colab import userdata
userdata.get('Secret_key')
import os # Import the os module to access environment variables
openai_api_secret_name = 'Secret_key'
OPENAI_API_KEY=userdata.get(openai_api_secret_name)
client = OpenAI(api_key=OPENAI_API_KEY)
my_assistant = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    name="Math Tutor",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo",
)
print(my_assistant)

Assistant(id='asst_2sOVfwfyjqTC4UjzITBIbR3L', created_at=1720429210, description=None, instructions='You are a personal math tutor. When asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4-turbo', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)


In [15]:
my_assistant_id = "asst_2sOVfwfyjqTC4UjzITBIbR3L"  # 실제 어시스턴트 ID로 변경

def ask_math_question(question):
    thread = client.beta.threads.create()  # 새 스레드 생성

    # 질문 메시지 추가
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )

    # 어시스턴트 실행
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=my_assistant_id
    )

    # 실행 완료될 때까지 대기 (필요한 경우 timeout 설정)
    import time
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        time.sleep(1)  # 1초 대기

    # 답변 가져오기
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    answer = messages.data[0].content[0].text.value  # 답변 추출

    return answer

# 질문 예시
question = "2차 방정식 x^2 + 2x - 3 = 0의 해를 구해주고 풀이과정을 작성해줘."
answer = ask_math_question(question)
print(answer)

2차 방정식 \(x^2 + 2x - 3 = 0\) 의 해는 \(x = 1\) 과 \(x = -3\) 입니다.

### 풀이 과정
1. 주어진 2차 방정식은 \(x^2 + 2x - 3 = 0\) 입니다.
2. 근의 공식에 사용되는 변수 \(a\), \(b\), \(c\) 는 각각 1, 2, -3 입니다.
3. 근의 공식은 다음과 같습니다:
   \[ x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} \]
4. 이때 판별식 \( \Delta = b^2 - 4ac \) 을 계산하면,
   \[ \Delta = 2^2 - 4 \cdot 1 \cdot (-3) = 4 + 12 = 16 \]
5. 판별식의 제곱근 \( \sqrt{\Delta} \) 은 4입니다.
6. 그러므로, 두 근은 다음과 같이 계산됩니다:
   \[
   x_1 = \frac{-2 + 4}{2 \times 1} = 1, \quad
   x_2 = \frac{-2 - 4}{2 \times 1} = -3
   \]

따라서, 이 방정식의 두 해는 1과 -3입니다.


In [1]:
#@title Configure OpenAI API key

# access your OpenAI API key

# installing llmx first isn't necessary but avoids a confusing error when installing openai
!pip install -q llmx
!pip install -q openai
from openai import OpenAI
from google.colab import userdata
openai_api_secret_name = 'Secret_key'  # @param {type: "string"}

try:
  OPENAI_API_KEY=userdata.get(openai_api_secret_name)
  client = OpenAI(
    api_key=OPENAI_API_KEY
  )
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found\n\nThis expects you to create a secret named {openai_api_secret_name} in Colab\n\nVisit https://platform.openai.com/api-keys to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {openai_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {openai_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {openai_api_secret_name} stored in Colab and it's a valid key from https://platform.openai.com/api-keys")
  raise e


In [9]:
#@title Ask a question!

text = '구구단 계산기 작성 파이썬 코드 작성해줘' # @param {type:"string"}

# ask Gemini
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])

response = chat.send_message('%s -- Please answer as concisely as you can, avoiding any extra conversation or text' % text)
gemini_response = response.text

# ask ChatGPT
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": '%s -- Please answer as concisely as you can, avoiding any extra conversation or text' % text}
  ]
)

openai_response = completion.choices[0].message.content

# render the diff

# importing these every execution is unnecessary but avoids another notebook cell
from IPython.display import HTML
import difflib

# omit the legend to slim down the UI
difflib.HtmlDiff._legend = ''

HTML(difflib.HtmlDiff().make_file(gemini_response.splitlines(), openai_response.splitlines(), 'Gemini', 'ChatGPT'))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
